In [ ]:
import requests
import json
import csv
import configparser

### Retrieve the environment you are connecting to.

We define a function that asks for the name of the FOLIO environment, which needs to match a section in the config file.

Then, we use the name to create the header values to be used in making your API request to FOLIO.

In [ ]:
def singleenvironment(configNames):
    environmentname = input("Which environment are we uploading reserves to? ")
    if environmentname in configNames:
        return environmentname
    else:
        sys.exit("can't recognize environment name!")

## Read in the environments you might be using from the config-template.ini file

In [ ]:
envConfig = configparser.ConfigParser()
envConfig.read('config-template.ini')

In [ ]:
uploadenvironment = singleenvironment(envConfig.sections())

### Define the header values to be used in making your API request to FOLIO.

In [ ]:
uploadurl = envConfig[uploadenvironment]['okapi_url']

fetchHeaders = {
    'x-okapi-tenant': envConfig[uploadenvironment]['tenant_id'],
    'x-okapi-token': envConfig[uploadenvironment]['password']
}
postHeaders = {
    'x-okapi-tenant': envConfig[uploadenvironment]['tenant_id'],
    'x-okapi-token': envConfig[uploadenvironment]['password'],
    'Content-Type': 'application/json'
}

### Create a reserve

The FOLIO API to create a reserve is **POST your-okapi-url/coursereserves/reserves** with the required fields barcode and course listing.

The goal is to take an input file with a list of barcodes, one on each line, and construct a JSON object for each barcode like this:

{
    "courseListingId": *"yourcourselistingid"*,
    "copiedItem": {
        "barcode": *"itembarcode"*
        }
}

Then, you can call the API for each item until all of them are sent.

**Step 1**: Create the URL for the reserve API call.

In [ ]:
uploadReserveUrl = '{}{}'.format(uploadurl, "/coursereserves/reserves")

**Step 2**: Ask for the course listing ID.

In [ ]:
courseListingIdForLoad = input("Please provide the UUID for the course listing we are adding items to ")

**Step 3**: Open your file of incoming barcodes.

In [ ]:
initialFile = open('barcodes.csv', newline='', encoding='utf-8-sig')
barcode_file = csv.reader(initialFile, dialect='excel')

**Step 4**: Construct and send the data payload to FOLIO for each reserve item.

In [ ]:
for each in barcode_file:
    payload = {}
    copiedItem = {}
    payload['courseListingId'] = courseListingIdForLoad
    copiedItem['barcode'] = each[0]
    payload['copiedItem'] = copiedItem
    requestPayload = json.dumps(payload)
    sendReserve = requests.post(uploadReserveUrl, headers=postHeaders, data=requestPayload)